## PDF Parsing

In [17]:
import sys
import os 
print(os.getcwd())

#sys.path.append('../src/')
#import src.ReadPDF as pdf_reader
import src.mapping as mapping
print(mapping.__file__)
# print(mapping in globals())
#sys.path.append('../src/classes')
import src.classes
from src.classes.PDFReader import PDFReader as pdf_reader
#import inspect
#print(inspect.getmodule(pdf_reader))
#print(sys.path)
from src.classes.Prompter import Prompter
#print(inspect.getmodule(Prompter))
import importlib
#print(dir(sys.modules))
#importlib.reload(pdf_reader)
# importlib.reload(ReadPDF)


/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/mapping.py


In [18]:
print('src.classes.PDFReader' in sys.modules)
for key in sorted(sys.modules):
    #print(type(key))
    pass
#importlib.reload(pdf_reader)
importlib.reload(src.classes.PDFReader)
#importlib.reload(pdf_reader)

True


<module 'src.classes.PDFReader' from '/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/PDFReader.py'>

In [19]:
# Extract by Section
#pdf_path = "20-301704.3.pdf" # ESA BL Data row 5736 - 6666
pdf_path = "4444_East.pdf" # ESA BL Data row 5736 - 6666

In [20]:
pdf_path2 = "20-301704.3.pdf" # ESA BL Data row 5736 - 6666


In [21]:
# Extract by Subsection
subsection_dict = pdf_reader().process_file(pdf_path)
#print(subsection_dict.keys()) #['1'][0].display_text())
#one_subsection = pdf_reader().extract_subsections(subsection_dict, 1, 1)
#print(one_subsection)

here
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/../../data/raw/4444_East.pdf
extracing by section .......................................................................................
Section Map:  {'1': {'1.0': ' 1.0 INTRODUCTION Partner Engineering and Science, Inc. (Partner) has performed a Phase I Environmental Site Assessment (ESA) in conformance with the scope and limitations of ASTM Standard Practice E1527-13 and the Environmental Protection Agency Standards and Practices for All Appropriate Inquiries (AAI) (40 CFR Part 312) for the property located at 4444 East 26th Street in Vernon, Los Angeles County, California (the “subject property”).  Any exceptions to, or deletions from, this scope of work are described in the report.', '1.1': ' 1.1 Purpose The purpose of this ESA is to identify existing or potential Recognized Environmental Conditions (as defined by ASTM Standard E-1527-13) affecting the subject property that: 1) constitute or result in 

In [22]:
subsection_dict = pdf_reader().process_file(pdf_path2)



here
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/../../data/raw/20-301704.3.pdf
extracing by section .......................................................................................
Section Map:  {'1': {'1.0': ' 1.0    INTRODUCTION Partner Engineering and Science, Inc. (Partner) has performed a Phase I Environmental Site Assessment (ESA) in conformance with the scope and limitations of ASTM Practice E1527-21 and the Environmental Protection Agency Standards and Practices for All Appropriate Inquiries (AAI) (40 CFR Part 312) for the property located at 115 West Anapamu Street in Santa Barbara, Santa Barbara County, California (the “subject property”).  Any exceptions to, or deletions from, this scope of work are described in the report.', '1.1': ' 1.1    Purpose The purpose of this ESA is to identify existing or potential Recognized Environmental Conditions (as defined by ASTM Standard E1527-21) affecting the subject property that: 1) constitute or r

In [23]:
# Extract by Subsection
parser = pdf_reader()
subsection_dict = parser.process_file(pdf_path)
#subsection_dict

here
/Users/natalieperrochon/CS180/new_clone/Partner-ESI-ML-Report/src/classes/../../data/raw/4444_East.pdf
extracing by section .......................................................................................
Section Map:  {'1': {'1.0': ' 1.0 INTRODUCTION Partner Engineering and Science, Inc. (Partner) has performed a Phase I Environmental Site Assessment (ESA) in conformance with the scope and limitations of ASTM Standard Practice E1527-13 and the Environmental Protection Agency Standards and Practices for All Appropriate Inquiries (AAI) (40 CFR Part 312) for the property located at 4444 East 26th Street in Vernon, Los Angeles County, California (the “subject property”).  Any exceptions to, or deletions from, this scope of work are described in the report.', '1.1': ' 1.1 Purpose The purpose of this ESA is to identify existing or potential Recognized Environmental Conditions (as defined by ASTM Standard E-1527-13) affecting the subject property that: 1) constitute or result in 

## LLM Setup + Prompt

In [24]:
prompter = Prompter()

In [25]:
%%time
#response = prompter.invoke('Name of Assessor/Appraisal District Agency', one_subsection)
datafields = mapping.execute()
#datafields.items()
# for subsection in sorted(datafields.items()):
#     print(f"-----------Section_num: {subsection}--------")
#     print(subsection, datafields.get_section_fields(subsection))
    
    # response = prompter.invoke(datafields[subsection.items()[1], 
#print(response)

TypeError: 'module' object is not callable

In [26]:
#print(datafields.items())

In [ ]:
from langchain_community.llms import Ollama
llm = Ollama(model="phi", temperature=0, num_predict=40, top_k=5, top_p=.3, mirostat_tau=0, format="json")

In [27]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.output_parsers import JsonOutputParser

output_parser = StrOutputParser()
json_parser = JsonOutputParser()

In [28]:
import json

In [16]:
%%time
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template("""Retrieve the datafield defined in input within the context given.
Anything legal can be found in the context.
If you cannot answer the question with the context, respond with N/A.

<context>
{context}
</context>

{input} is the prompt.
Again, if you cannot answer the question respond N/A.
Simply respond with only the answer that matches the datafield.
There must only be one JSON object.
Do not include the datafield in your response. Keep the key for the json exactly the same as the input.
""")

document_chain = create_stuff_documents_chain(llm, prompt, output_parser=json_parser)

CPU times: user 834 µs, sys: 1e+03 ns, total: 835 µs
Wall time: 842 µs


In [29]:
from langchain_community.llms import Ollama
system = """You are an assistant reading through a PDF and giving only the answers found within it. If you do not know an answer, you respond N/A."""
template = """[INST] {{ .System }} {{ .Prompt}} [/INST]"""
stop_tokens = ["[INST]", "[/INST]"]
llm = Ollama(model="medllama2", system=system, template=template,
             stop=stop_tokens, num_predict=228, top_k=0, top_p=0)


## Responses

In [30]:
fields_dict = {'ets_2p_32a': 'Name of Assessor/Appraisal District Agency', 
               'ets_2p_33a': 'Property Legal Description', 
               'ets_2p_34a': 'Property Owner Name'}
#               'ets_2p_35a': "When did the owner acquire the property?",
 #              "ets_2p_36a": "What was the source of the property owner name and acquisition date?"}

In [31]:
#mappings = mapping.execute()

In [32]:
#mappings
# {subsection_num: {field: question}}

In [33]:
# print(datafields.items())

In [34]:
%%time
from langchain_core.documents import Document
answers_dict = {}

for items in datafields.items():
    try:
    
        item = {items[0]: items[1]} 
        # Item is of shape : {'1.1' : {'ets_1p_32a': 'Purpose:', 'esa_foia_1': 'ESA Jurisdiction 1', 
        #                     'esa_foia_contact_1': 'ESA Jurisdiction Contact 1', 'ESA Jurisdiction Contact 3'}}
        # print("ITEM IS " , item)
    except Exception as e:
        print('ERROR')
        print(items)
        print(e)
        print()
    for subsection in item:
        print("SUBSECTION:" ,subsection)
        print(subsection == '.1')
        section_number = subsection.split('.')[0]
        print(section_number)
        # TAKE THIS OUT: JUST FOR DEMO PURPOSES:
        if '5.1.1' in subsection:
            for field in item[subsection]:
                print(field)
                # print(item[subsection][field])
                # for key in subsection_dict.keys():
                #     print('.' + key + '.')
                #     print(type(key))
                    
                #print(subsection_dict.keys())
               # section = subsection.
                #print(' 3' in subsection_dict.keys())
                #print(item[subsection][field])
                try:
                    print(subsection_dict[section_number].keys())
                    subsection_context = subsection_dict[section_number][subsection]
                    #print("SUBSECTION CONTEXT: ", subsection_context)
                    print("INPUT QUESTION IS ", item[subsection][field])
                    output_dict = document_chain.invoke({
                        "input": item[subsection][field],
                        "context": [Document(page_content=subsection_context)]
                    })
                    key = list(output_dict.keys())[0]
                    real_output_dict = {}
                    answers_dict[field] = output_dict[key]
                    print('MODEL OUTPUTS' , output_dict)
                    print('PROCESSED OUTPUT' , real_output_dict)
                    #print(output_dict.keys[0])
                except Exception as e:
                    print('exception: ', e)
                #answers_dict[field] = output_dict[item[subsection][field]]
                #print(answers_dict[field])
print("answers dict is ", answers_dict)
with open("data/processed/field.json", "w") as outfile: 
    json.dump(answers_dict, outfile)

NameError: name 'datafields' is not defined

In [22]:
%%time
from langchain_core.documents import Document

print(type(document_chain.invoke({
    "input": "Property Legal Description",
    "context": [Document(page_content=one_subsection)]
})))


KeyboardInterrupt



In [23]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "Property Owner Name",
    "context": [Document(page_content=one_subsection)]
})


KeyboardInterrupt



In [24]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "When did the owner acquire the property?",
    "context": [Document(page_content=one_subsection)]
})

NameError: name 'one_subsection' is not defined

In [ ]:
%%time
from langchain_core.documents import Document

document_chain.invoke({
    "input": "What was the source of the property owner name and acquisition date?",
    "context": [Document(page_content=one_subsection)]
})